# **BERTopic** - Dynamic Topic Modeling with Twees hashtags: #manunited, #manchesterunited,#MUFC


In this tutorial we will be using Dynamic Topic Modeling with BERTopic to visualize how topics in Tweets have evolved over time. These topics will be visualized and thoroughly explored. 

## Dynamic Topic Models
Dynamic topic models can be used to analyze the evolution of topics of a collection of documents over time. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this model we will need Tweets as Data. How to get data will be captured in a other notebook.

In this notebook we will do two DF's. One with stopwords, and one without stopwords. We will compare the result

In [ ]:
import re
import pandas as pd
from datetime import datetime

In [ ]:
tweets_no_stopwords = pd.read_csv('/content/drive/MyDrive/Interim/_Post_Twitter_all_Hashtags_with_matches_clean_removed_stopwords_no_lem.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (14,15,16,17,18,19,20,21,22,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
tweets_no_stopwords= tweets_no_stopwords.dropna(subset = ['Tweet_clean'])

In [ ]:
# create a list of all tweets
tweets_no_stopwords_lst = tweets_no_stopwords.Tweet_clean.to_list()
tweets_no_stopwords_lst[5123]
timestamps = tweets_no_stopwords.Date_Created.to_list()

In [ ]:
tweets_no_stopwords_lst[5123]

'Full support ManUtd'

# **Dynamic Topic Modeling**


## Basic Topic Model
To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all tweets. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those tweets. 

In [ ]:
##%%time
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=35, verbose=True)
topics, _ = topic_model.fit_transform(tweets_no_stopwords_lst)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/33593 [00:00<?, ?it/s]

2022-11-09 11:06:54,159 - BERTopic - Transformed documents to Embeddings
2022-11-09 13:10:38,429 - BERTopic - Reduced dimensionality
2022-11-09 13:14:11,530 - BERTopic - Clustered reduced embeddings


We can then extract most frequent topics:

In [ ]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,-1,520685,-1_jong_frenkie_barcelona_ronaldo
1,0,6899,0_liverpool_liverpoolfc_liverpools_beat
2,1,6234,1_signings_sign_signing_signed
3,2,5861,2_jadon_sancho_sanchooo_sanchos
4,3,5696,3_haag_ten_hag_hags
5,4,5419,4_oh_accurate_interesting_love
6,5,5105,5_gea_david_geas_de
7,6,4328,6_brighton_bhamun_hove_albion
8,7,4216,7_fabrizioromano_caughtoffside_gettyimages_fab
9,8,4109,8_henderson_dean_forest_nottingham


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(4)

[('oh', 0.004259714562970829),
 ('accurate', 0.0035216235022884276),
 ('interesting', 0.002996559234843513),
 ('love', 0.0027784602787626483),
 ('lol', 0.0025949206970104543),
 ('go', 0.0025273435763190854),
 ('fuck', 0.002379300111827104),
 ('exactly', 0.002358197869057448),
 ('wow', 0.002353931228478137),
 ('come', 0.002229340020141025)]

# Save the model #

In [ ]:
import pickle


In [ ]:
# save the model to disk
filename = 'finalized_model_BERTopic_1M_DB_no_Stopwords_no_lem.sav'
pickle.dump(topic_model, open(filename, 'wb'))

NameError: ignored

# Load the model

In [ ]:
# load the model from disk
filename = 'finalized_model_BERTopic_1M_DB_no_Stopwords_no_lem.sav'
loaded_model = pickle.load(open(filename, 'rb'))
freq = loaded_model.get_topic_info(); freq.head(10)


FileNotFoundError: ignored


We can visualize the basic topics that were created with the Intertopic Distance Map. This allows us to judge visually whether the basic topics are sufficient before proceeding to creating the topics over time.

In [ ]:
fig = topic_model.visualize_topics(); fig

KeyboardInterrupt: ignored

## Topics over Time
Before we start with the Dynamic Topic Modeling step, it is important that you are satisfied with the topics that were created previously. We are going to be using those specific topics as a base for Dynamic Topic Modeling. 

Thus, this step will essentially show you how the topics that were defined previously have evolved over time. 

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20. 


In [ ]:
topics_over_time = topic_model.topics_over_time(docs=tweets_no_stopwords_lst, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

20it [20:29, 61.48s/it]


## Visualize Topics over Time
After having created our `topics_over_time`, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension. 

To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time *t* differs from the global topic representation. 

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=50)